In [1]:
# Servo Control
import time
import wiringpi

class Servo(object):
    def __init__(self, pin):
        pass
    
    def hardware_init(self):
        # use 'GPIO naming'
        wiringpi.wiringPiSetupGpio()        
        # set #18 to be a PWM output
        wiringpi.pinMode(18, wiringpi.GPIO.PWM_OUTPUT)

        # set the PWM mode to milliseconds stype
        wiringpi.pwmSetMode(wiringpi.GPIO.PWM_MODE_MS)

        # divide down clock
        wiringpi.pwmSetClock(192)
        wiringpi.pwmSetRange(2000)

    def test(self):
        delay_period = 0.01

        while True:
            for pulse in range(50, 250, 1):
                wiringpi.pwmWrite(18, pulse)
                time.sleep(delay_period)
            for pulse in range(250, 50, -1):
                wiringpi.pwmWrite(18, pulse)
                time.sleep(delay_period)

In [14]:
class MeArm(object):
    def __init__(self):
        self.L1 = 0.0 #Shoulder to elbow length
        self.L2 = 0.0 #Elbow to wrist length
        self.L3 = 0.0 #Wrist to hand length
        self.servos = {"base":Servo(1),
                      "shoulder":Servo(2),
                      "elbow":Servo(3),
                      "grip":Servo(4)}
        
        self.IK = IK()

    def base(self, angle):
        print "Moving base to:", angle
        self.servos["base"]
    
    def shoulder(self, angle):
        print "Moving shoulder to:", angle        
        self.servos["shoulder"]
    
    def elbow(self, angle):
        print "Moving elbow to:", angle
        self.servos["elbow"]
    
    def gripper(self, angle):
        print "Moving gripper to:", angle
        self.servos["grip"]    
    
    def openGripper(self):
        print "Grip open"
    
    def closeGripper(self):
        print "Grip close"
    
    def clap(self):
        self.openGripper(); self.closeGripper()
        self.openGripper(); self.closeGripper()
        self.openGripper()        
    
    def gotoPoint(self, x, y, z):
        print "Moving to:",(x,y,z)
        if self.IK.isReachable(x,y,z):
            a,b,g = self.IK.positionToJoints(x,y,z) 
            self.base(a)
            self.shoulder(b)
            self.elbow(g)
    
    

arm = MeArm()
arm.clap()
#Go up and left to grab something
arm.gotoPoint(-80,100,140); 
arm.closeGripper();
#Go down, forward and right to drop it
arm.gotoPoint(70,200,10);
arm.openGripper();
#Back to start position
arm.gotoPoint(0,100,50);

Grip open
Grip close
Grip open
Grip close
Grip open
Moving to: (-80, 100, 140)
Moving base to: 0.0
Moving shoulder to: 0.0
Moving elbow to: 0.0
Grip close
Moving to: (70, 200, 10)
Moving base to: 0.0
Moving shoulder to: 0.0
Moving elbow to: 0.0
Grip open
Moving to: (0, 100, 50)
Moving base to: 0.0
Moving shoulder to: 0.0
Moving elbow to: 0.0


In [12]:
class FK(object):
    def jointsToPosition(self, a,b,g):
        x,y,z = .0,.0,.0
        return x,y,z

In [13]:
class IK(object):
    def isReachable(self, x,y,z):
        return True
    
    def positionToJoints(self, x,y,z):
        a,b,g = .0,.0,.0
        return a,b,g